In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from collections import Counter
import gc
import time
import psutil
import os

"""
1. pandas (pd)
   - Used for structured/tabular data manipulation.
   - Likely used later for reading protein datasets (e.g., CSV with sequences and GO terms).
   - Provides functions like:
       - read_csv(), sort_values(), unique(), groupby(), reset_index()

2. numpy (np)
   - Used for numerical operations.
   - Deep learning models require numerical arrays.
   - Useful for:
       - vector operations, encoding labels, reshaping data

3. Dataset (from HuggingFace datasets library)
   - Converts data into an optimized format for training.
   - Often used with transformers or large NLP models.
   - If not using HuggingFace Trainer or tokenizers later, this import may be unnecessary.

4. Counter
   - Counts frequency of elements in a list.
   - Very useful for:
       - Counting GO term frequencies, Detecting class imbalance

5. gc (Garbage Collector)
   - Frees memory manually using gc.collect()
   - Important when handling large biological datasets.

6. time
   - Used to measure execution time.

7. psutil
   - Tracks memory usage of current process, Useful for monitoring RAM consumption.

8. os
   - Used for operating system utilities.
   - Here it is used to get current process ID.
"""

start_time = time.time()
process = psutil.Process(os.getpid())

"""
1. time.time()
   - Returns current time in seconds since epoch.
   - Used to calculate total runtime later: total_time = time.time() - start_time

2. os.getpid()
   - Gets the process ID of the current Python program.

3. psutil.Process(pid)
   - Attaches a memory-monitoring object to this process.
   - Allows access to:
       - memory usage, CPU usage, other process stats
"""

def print_memory_usage(step_name):
    mem_info = process.memory_info()
    mem_gb = mem_info.rss / (1024 ** 3)

    print(f"{step_name} - Memory Usage: {mem_gb:.2f} GB")
    return mem_gb

"""
Function Purpose:
This function checks how much RAM the program is currently using.

1. process.memory_info()
   - Retrieves memory statistics for current process.
   - Returns an object containing:
       - rss (Resident Set Size)
       - vms (Virtual memory size)

2. mem_info.rss
   - RSS = actual physical memory being used.
   - This is the most relevant measure for memory usage.

3. (1024 ** 3)
   - Converts bytes to gigabytes.

4. print(f"{step_name} - Memory Usage: {mem_gb:.2f} GB")
   - Displays step name AND memory.
   - This fixes earlier version where step_name was unused.
"""

def print_step_header(step_num, step_name):
    separator = "=" * 60

    print("\n" + separator)
    print(f"STEP {step_num}: {step_name}")
    print(separator)

"""
1. "=" * 60
   - Repeats "=" 60 times.
   - Creates a clear visual separator.

2. print("\n" + separator)
   - Adds spacing before new step.

3. f"STEP {step_num}: {step_name}"
   - Displays structured step numbering.

Why this is useful:
When processing protein datasets, multiple stages exist:
    - Loading data, Cleaning, Encoding, Training, Evaluation

This prevents confusion during long runs.
"""

initial_mem = print_memory_usage("Initial")

print("\nSTARTING PROTEIN GO TERM PROCESSING")
print(f"Start Time: {time.strftime('%H:%M:%S')}")
print(f"Initial Memory: {initial_mem:.2f} GB")

"""
1. initial_mem = print_memory_usage("Initial")
   - Captures memory before processing begins.
   - Later we can compare:
         current_memory - initial_memory
   - Helps detect memory leaks.

2. time.strftime('%H:%M:%S')
   - Formats current time as Hour:Minute:Second.

3. Logging statements
   - Useful for reproducibility.
   - Important when running experiments overnight or on servers.
"""

c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initial - Memory Usage: 0.29 GB

STARTING PROTEIN GO TERM PROCESSING
Start Time: 22:54:16
Initial Memory: 0.29 GB


'\nEXPLANATION:\n\n1. initial_mem = print_memory_usage("Initial")\n   - Captures memory before processing begins.\n   - Later we can compare:\n         current_memory - initial_memory\n   - Helps detect memory leaks.\n\n2. time.strftime(\'%H:%M:%S\')\n   - Formats current time as Hour:Minute:Second.\n\n3. Logging statements\n   - Useful for reproducibility.\n   - Important when running experiments overnight or on servers.\n'

In [ ]:
from Bio import SeqIO

"""
Bio.SeqIO is a module from the Biopython library used for reading
biological sequence files such as FASTA, GenBank, etc.

SeqIO can:
- Parse FASTA files automatically
- Extract IDs and sequences cleanly
- Avoid manual string parsing
"""

tsv_path = "C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/raw/train/uniprotkb_AND_reviewed_true_AND_protein_2025_12_27.tsv"
labels_df = pd.read_csv(tsv_path, sep='\t')

"""
1. pd.read_csv(tsv_path, sep='\t')

   - read_csv() loads tabular data into a pandas DataFrame.
   - sep='\t' tells pandas that the file is TAB-separated.
   - TSV = Tab Separated Values.

labels_df is a DataFrame (rows × columns).
This DataFrame will later be merged with sequences.
"""

fasta_path = "C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/raw/train/uniprotkb_AND_reviewed_true_AND_protein_2025_12_27.fasta"
sequence_dict = {}

for record in SeqIO.parse(fasta_path, "fasta"):
    # UniProt FASTA headers look like:
    # sp|P12345|PROTEIN_NAME
    # We extract the middle ID (P12345)

    header = record.id
    if "|" in header:
        protein_id = header.split("|")[1]
    else:
        protein_id = header.split()[0]
    sequence_dict[protein_id] = str(record.seq)

"""
1. SeqIO.parse(fasta_path, "fasta")

   - Reads FASTA file safely.
   - Automatically handles:
        - Multi-line sequences
        - Header parsing
        - End-of-file cases

2. record.id
   - Returns FASTA header identifier.
   - Example:
        sp|P12345|PROTEIN_NAME

3. header.split("|")[1]
   - Splits string at "|".
   - Index 1 extracts middle ID (P12345).
   - This matches the original logic.

4. sequence_dict[protein_id] = str(record.seq)
   - record.seq is a Seq object.
   - Convert to string for easier handling later.

Result:
sequence_dict:
    {
        "P12345": "MTEYKLVVVGAGGVGKS...",
        ...
    }
"""

print(f"TSV entries: {len(labels_df)}")
print(f"FASTA sequences: {len(sequence_dict)}")

"""
1. len(labels_df)
   - Returns number of rows in DataFrame.
   - Equivalent to number of labeled proteins.

2. len(sequence_dict)
   - Returns number of protein sequences parsed.

Why this is important:
We must verify that:
    number of labels ≈ number of sequences

If they differ significantly:
    - Some proteins may lack sequences
    - Some sequences may lack labels
    - IDs may not match

Later, we must ensure proper merging.
"""

TSV entries: 105951
FASTA sequences: 105951


"\nEXPLANATION:\n\n1. len(labels_df)\n   - Returns number of rows in DataFrame.\n   - Equivalent to number of labeled proteins.\n\n2. len(sequence_dict)\n   - Returns number of protein sequences parsed.\n\nWhy this is important:\nWe must verify that:\n    number of labels ≈ number of sequences\n\nIf they differ significantly:\n    - Some proteins may lack sequences\n    - Some sequences may lack labels\n    - IDs may not match\n\nLater, we must ensure proper merging.\n\nIMPORTANT NEXT STEP (very important for deep learning):\n\nWe must check:\n    set(labels_df['protein_id_column']) \n        ∩ \n    set(sequence_dict.keys())\n\nIf intersection is smaller than expected, we may silently lose data.\n"

In [ ]:
def parse_go_terms(go_string):
    """
    Convert a semicolon-separated GO string into a Python list.

    Example:
        Input  → "GO:0008150;GO:0003674"
        Output → ["GO:0008150", "GO:0003674"]

    This is required because:
    - The dataset stores multiple GO annotations in one cell.
    - Deep learning models cannot work with raw strings.
    - We must convert them into structured format (list).
    """

    if pd.isna(go_string) or go_string == "":
        return []

    """
    1. pd.isna(go_string)
       - Checks whether value is NaN (missing).
       - NaN can appear if a protein has no GO annotations.

    2. go_string == ""
       - Handles empty strings.
       - Sometimes TSV files store missing values as "" instead of NaN.

    Why return [] instead of None?
       - Returning an empty list keeps data type consistent.
       - Later, we will expect a LIST of GO terms.
       - Returning None would break later loops.

    Deep learning perspective:
       If we are doing multi-label classification,
       returning [] means this protein has no labels.
       We must later decide:
           - Remove such proteins or Keep them?
    """

    return [term.strip() for term in str(go_string).split(';')]

    """
    1. str(go_string)
       - Ensures the value is treated as string.
       - Defensive programming in case data type is unexpected.

    2. .split(';')
       - Splits string at each semicolon.
       - Example:
           "GO:0003953; GO:0007165"
       becomes:
           ["GO:0003953", " GO:0007165"]

    3. term.strip()
       - Removes leading/trailing whitespace.
       - Important because some entries have space after semicolon.
       - Example: " GO:0007165" → "GO:0007165"

    4. List comprehension
       - Efficient way to apply transformation to each term.
       - Cleaner than using a for-loop.

    FINAL RESULT: A Python list of GO term strings.

    IMPORTANT CONCEPTUAL NOTE:
        We are converting from:
            Multi-label string format
        into:
            Structured multi-label list format.

        This means the problem is NOT single-class classification, It is multi-label classification.
    """

labels_df['go_terms_list'] = labels_df['Gene Ontology IDs'].apply(parse_go_terms)

"""
1. labels_df['Gene Ontology IDs']
   - Selects the column containing raw GO strings.

2. .apply(parse_go_terms)
   - Applies the function to EACH row.
   - Equivalent to:
         for each row:
             parse_go_terms(value)
   But much cleaner and vectorized.

3. labels_df['go_terms_list'] = ...
   - Creates a NEW column.
   - Does not overwrite original data.
   - Good practice for traceability.

Now your DataFrame contains:

Column: 'Gene Ontology IDs'
    → Raw string format

Column: 'go_terms_list'
    → Python list format

Example:

Before:
    "GO:0003953; GO:0007165"
After:
    ["GO:0003953", "GO:0007165"]

This structured list is REQUIRED for:
    - Counting frequencies (Counter), Building label vocabulary, Multi-hot encoding, Training neural networks

    IMPORTANT:

Because each protein can have MULTIPLE GO terms,
the task is: MULTI-LABEL CLASSIFICATION

This changes everything downstream:

1. We CANNOT use:
       CrossEntropyLoss, Because that assumes ONE label per sample.

2. We MUST use:
       BCEWithLogitsLoss (if PyTorch) or BinaryCrossentropy (if Keras)

3. The output layer must be:
       number_of_unique_GO_terms neurons

4. Final activation should be:
       Sigmoid (not Softmax)

Softmax = probabilities sum to 1 (single-label)
Sigmoid = independent probabilities per class (multi-label)
"""

'\nIMPORTANT:\n\nBecause each protein can have MULTIPLE GO terms,\nthe task is:\n    MULTI-LABEL CLASSIFICATION\n\nThis changes everything downstream:\n\n1. We CANNOT use:\n       CrossEntropyLoss\n   Because that assumes ONE label per sample.\n\n2. We MUST use:\n       BCEWithLogitsLoss (if PyTorch)\n   or\n       BinaryCrossentropy (if Keras)\n\n3. The output layer must be:\n       number_of_unique_GO_terms neurons\n\n4. Final activation should be:\n       Sigmoid (not Softmax)\n\nSoftmax = probabilities sum to 1 (single-label)\nSigmoid = independent probabilities per class (multi-label)\n'

In [5]:
labels_df.head()

,Entry,Entry Name,Protein names,Organism,Sequence,Gene Ontology IDs,go_terms_list
0,A0A009IHW8,ABTIR_ACIB9,2' cyclic ADP-D-ribose synthase AbTIR (2'cADPR...,Acinetobacter baumannii (strain 1295743),MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,GO:0003953; GO:0007165; GO:0019677; GO:0050135...,"[GO:0003953, GO:0007165, GO:0019677, GO:005013..."
1,A0A023I7E1,ENG1_RHIMI,"Glucan endo-1,3-beta-D-glucosidase 1 (Endo-1,3...",Rhizomucor miehei,MRFQVIVAAATITMITSYIPGVASQSTSDGDDLFVPVSNFDPKSIF...,GO:0000272; GO:0005576; GO:0042973; GO:0052861...,"[GO:0000272, GO:0005576, GO:0042973, GO:005286..."
2,A0A024B7W1,POLG_ZIKVF,Genome polyprotein [Cleaved into: Capsid prote...,Zika virus (isolate ZIKV/Human/French Polynesi...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,GO:0003724; GO:0003725; GO:0003968; GO:0004252...,"[GO:0003724, GO:0003725, GO:0003968, GO:000425..."
3,A0A024RXP8,GUX1_HYPJR,"Exoglucanase 1 (EC 3.2.1.91) (1,4-beta-cellobi...",Hypocrea jecorina (strain ATCC 56765 / BCRC 32...,MYRKLAVISAFLATARAQSACTLQSETHPPLTWQKCSSGGTCTQQT...,GO:0005576; GO:0016162; GO:0030245; GO:0030248,"[GO:0005576, GO:0016162, GO:0030245, GO:0030248]"
4,A0A024SC78,CUTI1_HYPJR,Cutinase (EC 3.1.1.74),Hypocrea jecorina (strain ATCC 56765 / BCRC 32...,MRSLAILTTLLAGHAFAYPKPAPQSVNRRDWPSINEFLSELAKVMP...,GO:0005576; GO:0016052; GO:0050525,"[GO:0005576, GO:0016052, GO:0050525]"


In [ ]:
filtered_df = labels_df[labels_df['Entry'].isin(sequence_dict.keys())].copy()

"""
In this section, we perform a vectorized matching operation between:

    - Protein accession IDs from the label DataFrame
    - Keys of sequence_dict (which contain available FASTA sequences)

Step-by-step explanation:

1. labels_df['Entry']
   - This selects the column containing UniProt accession IDs.
   - Each value represents one protein.

2. .isin(sequence_dict.keys())
   - The isin() function checks whether each element of the column
     exists inside a given collection.
   - Here, the collection is sequence_dict.keys(), which contains
     all protein IDs parsed from the FASTA file.

   Conceptually, this creates a Boolean mask:
        True  → accession exists in sequence_dict
        False → accession does not exist

3. labels_df[ ... ]
   - We use this Boolean mask to filter rows.
   - This keeps only proteins that have corresponding sequences.

4. .copy()
   - Creates a deep copy of the filtered data.
   - This prevents SettingWithCopyWarning later.
   - It ensures modifications do not affect the original DataFrame.

Conceptual interpretation:

This operation is equivalent to performing an INNER JOIN between:
    labels_df (TSV data)
and
    sequence_dict (FASTA data)

However, instead of looping row-by-row, we use pandas' vectorized operations, which are significantly faster and more memory-efficient.
"""

filtered_df['sequence'] = filtered_df['Entry'].map(sequence_dict)

"""
Now that we have filtered proteins that exist in sequence_dict, we attach their sequences directly into the DataFrame.

1. .map(sequence_dict)

   - The map() function replaces each value in a Series
     using a dictionary lookup.
   - For each accession ID in filtered_df['Entry']:
         sequence_dict[accession] is retrieved.

   Example:
        If accession = "A0A009IHW8"
        Then:
            sequence_dict["A0A009IHW8"] → "MSLEQKKGADIISKIL..."

2. filtered_df['sequence'] = ...
   - Creates a new column called 'sequence'
   - Each row now contains:
         accession
         original metadata
         parsed GO term list
         full amino acid sequence

Why this is efficient:

- Dictionary lookup is O(1) on average.
- No explicit Python loops.
- Fully vectorized within pandas.
"""

train_df = filtered_df[['Entry', 'sequence', 'go_terms_list', 'Organism']].rename(
    columns={
        'Entry': 'accession',
        'go_terms_list': 'go_terms',
        'Organism': 'organism'
    }
)

"""
In this section, we reduce the DataFrame to only essential columns
required for model training.

1. filtered_df[['Entry', 'sequence', 'go_terms_list', 'Organism']]

   - Selects only necessary columns.
   - Removes irrelevant metadata (protein names, etc.).
   - This reduces memory usage and improves clarity.

2. .rename(columns={...})

   - Renames columns to cleaner, standardized names.
   - 'Entry' → 'accession'
   - 'go_terms_list' → 'go_terms'
   - 'Organism' → 'organism'

Why renaming is important:

- Improves readability
- Makes downstream code cleaner
- Avoids repeatedly referencing long column names

After this step, train_df contains:
   accession | sequence | go_terms | organism
This becomes our core training dataset.
"""

print(f"Matched proteins: {len(train_df)}")

"""
1. len(train_df)

   - Returns number of successfully matched proteins.
   - This equals the number of proteins that have:
         ✔ GO annotations
         ✔ Available amino acid sequence

Sanity check:

This number should be close to:
        min(len(labels_df), len(sequence_dict))

If it is significantly lower,
this indicates ID mismatches between TSV and FASTA files.

This step confirms integrity of data merging.
"""

output_path = "C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/processed/training_data_combined.csv"
train_df.to_csv(output_path, index=False)
print(f"Saved to: {output_path}")

"""
1. to_csv(output_path, index=False)

   - Saves processed dataset to disk.
   - index=False prevents saving row indices.
   - Ensures clean, reusable CSV file.

Why this step is important:

- FASTA parsing is computationally expensive.
- Merging datasets repeatedly wastes time.
- Saving intermediate dataset improves reproducibility.
- This file becomes the starting point for:
        tokenization
        label encoding
        model training

This concludes construction of the unified training dataset.
"""

Matched proteins: 105951
Saved to: C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/processed/training_data_combined.csv


'\nEXPLANATION:\n\n1. to_csv(output_path, index=False)\n\n   - Saves processed dataset to disk.\n   - index=False prevents saving row indices.\n   - Ensures clean, reusable CSV file.\n\nWhy this step is important:\n\n- FASTA parsing is computationally expensive.\n- Merging datasets repeatedly wastes time.\n- Saving intermediate dataset improves reproducibility.\n- This file becomes the starting point for:\n        tokenization\n        label encoding\n        model training\n\nThis concludes construction of the unified training dataset.\n'

In [7]:
train_df.head()

,accession,sequence,go_terms,organism
0,A0A009IHW8,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,"[GO:0003953, GO:0007165, GO:0019677, GO:005013...",Acinetobacter baumannii (strain 1295743)
1,A0A023I7E1,MRFQVIVAAATITMITSYIPGVASQSTSDGDDLFVPVSNFDPKSIF...,"[GO:0000272, GO:0005576, GO:0042973, GO:005286...",Rhizomucor miehei
2,A0A024B7W1,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,"[GO:0003724, GO:0003725, GO:0003968, GO:000425...",Zika virus (isolate ZIKV/Human/French Polynesi...
3,A0A024RXP8,MYRKLAVISAFLATARAQSACTLQSETHPPLTWQKCSSGGTCTQQT...,"[GO:0005576, GO:0016162, GO:0030245, GO:0030248]",Hypocrea jecorina (strain ATCC 56765 / BCRC 32...
4,A0A024SC78,MRSLAILTTLLAGHAFAYPKPAPQSVNRRDWPSINEFLSELAKVMP...,"[GO:0005576, GO:0016052, GO:0050525]",Hypocrea jecorina (strain ATCC 56765 / BCRC 32...


In [ ]:
print(f"Total proteins: {len(train_df)}")

"""
We begin by reporting the number of proteins available after merging sequences with GO annotations.
This number represents the total samples that can potentially be used for training.
This establishes baseline dataset size before label filtering.
"""

print("\nCollecting unique GO terms and computing frequencies...")
go_counts = Counter(term for go_list in train_df['go_terms'] for term in go_list)
all_go_terms = set(go_counts.keys())

print(f"Found {len(all_go_terms)} unique GO terms total")
mem1 = print_memory_usage("After collecting all terms")

"""
In this section, we compute BOTH:
    - The set of unique GO terms
    - The frequency of each GO term

Counter(term for go_list in train_df['go_terms'] for term in go_list)
Explanation:

- Outer loop iterates through each protein’s GO list.
- Inner loop iterates through each GO term in that list.
- Each term is fed into Counter.
- Counter automatically counts frequency.

This accomplishes:
    ✔ Unique term extraction, Frequency counting, Single pass over data

Computationally cleaner and more efficient.

all_go_terms = set(go_counts.keys())
- Extracts unique GO terms.
- Equivalent to previous manual set update.
This removes repetition while keeping identical results.
"""

ultra_general = {'GO:0008150', 'GO:0005575', 'GO:0003674'}

filtered_terms = [
    term for term, count in go_counts.most_common()
    if term not in ultra_general
]

"""
Gene Ontology has three root terms:

    GO:0008150 → Biological Process
    GO:0005575 → Cellular Component
    GO:0003674 → Molecular Function

These terms are extremely general, Nearly every protein inherits from them.

Including them would:
    - Add no learning value
    - Inflate class counts
    - Bias predictions
We remove them before selection.

go_counts.most_common()
- Returns list of (term, count) pairs sorted by descending frequency.
- This is important because:
    We want the most frequent terms first.

We extract only the term from each (term, count) tuple.
This preserves frequency order.

At this stage:
filtered_terms is ordered from most frequent → least frequent,
excluding ultra-general terms.
"""

top_n_frequent = 2000
top_go_frequent = filtered_terms[:top_n_frequent]

"""
We select the 700 most frequent GO terms.

Why frequent terms?

- Frequent classes have enough examples to train reliably.
- Rare classes can cause unstable gradients.
- Deep learning performs poorly on extremely sparse classes.

This ensures strong core signal in training.

This slice operation preserves order because:
most_common() already sorted by frequency.
"""

remaining_terms = filtered_terms[top_n_frequent:]

remaining_filtered = [
    term for term in remaining_terms
    if go_counts[term] >= 5
]
np.random.seed(42)

top_go_random = (
    np.random.choice(remaining_filtered, 761, replace=False).tolist()
    if len(remaining_filtered) >= 761
    else remaining_filtered
)

"""
We now add diversity by selecting additional terms.

Why random terms?

If we only select frequent terms:
    - Model becomes biased toward common functions.
    - Rare but meaningful functions are ignored.

We apply a learnability threshold:
    go_counts[term] >= 5

This ensures:
    - Each selected term appears at least 3 times.
    - Avoids selecting extremely rare terms (1–2 samples).

Random selection:
    np.random.choice(..., replace=False)

- replace=False ensures no duplicates.
- seed=42 ensures reproducibility.
- Every run produces identical selection.

If fewer than 761 valid terms exist,
we simply take all available ones.

This logic is identical to original code,
but slightly cleaner.
"""

top_go_terms = top_go_frequent + top_go_random

print(f"\nSelected {len(top_go_frequent)} frequent + "
      f"{len(top_go_random)} random = {len(top_go_terms)} total GO terms")

print(f"Most common term: {top_go_frequent[0]} "
      f"(appears {go_counts[top_go_frequent[0]]} times)")

if top_go_random:
    print(f"Sample random term: {top_go_random[0]} "
          f"(appears {go_counts[top_go_random[0]]} times)")

mem2 = print_memory_usage("After term selection")

print(f"Time elapsed: {time.time() - start_time:.1f} seconds")

"""
We combine:
    2000 frequent terms
+    761 random learnable terms
--------------------------------
=   2761 total GO terms

This defines the final output dimension of the model.

If we later perform multi-hot encoding,
the output layer size will be:
    len(top_go_terms)

This block also prints:
    - Frequency sanity check
    - Memory usage
    - Runtime tracking

This completes label vocabulary construction.
"""

Total proteins: 105951

Found 27615 unique GO terms total
After collecting all terms - Memory Usage: 0.55 GB

Selected 2000 frequent + 761 random = 2761 total GO terms
Most common term: GO:0005737 (appears 22662 times)
Sample random term: GO:0016311 (appears 62 times)
After term selection - Memory Usage: 0.55 GB
Time elapsed: 10.3 seconds


'\nEXPLANATION:\n\nWe combine:\n    2000 frequent terms\n+    761 random learnable terms\n--------------------------------\n=   2761 total GO terms\n\nThis defines the final output dimension of the model.\n\nIf we later perform multi-hot encoding,\nthe output layer size will be:\n    len(top_go_terms)\n\nThis block also prints:\n    - Frequency sanity check\n    - Memory usage\n    - Runtime tracking\n\nThis completes label vocabulary construction.\n'

In [9]:
# We create a mapping from GO term → column index, This allows fast assignment without looping over all labels repeatedly.
go_to_index = {go: idx for idx, go in enumerate(top_go_terms)}

num_proteins = len(train_df)
num_labels = len(top_go_terms)

print(f"Total proteins: {num_proteins:,}")
print(f"Total selected GO terms: {num_labels:,}")

# we create one full matrix at once using NumPy, Using int8 keeps memory small (1 byte per entry).

binary_matrix = np.zeros((num_proteins, num_labels), dtype=np.int8)

# At this stage, we already created:
# 1. go_to_index → mapping of GO term → column position
# 2. binary_matrix → a 2D NumPy array initialized with zeros
#
# Shape of binary_matrix:
# (number_of_proteins, number_of_selected_GO_terms)
#
# Each row represents ONE protein, Each column represents ONE GO term label.
# A value of 1 means the protein has that GO term, A value of 0 means it does not.
#
# We now populate this matrix.
fill_start_time = time.time()

# enumerate() gives us:
# protein_idx → integer row index (0, 1, 2, ...)
# go_list     → list of GO terms for that protein
#
# We use enumerate because we must know exactly, which row in the matrix corresponds to each protein.

for protein_idx, go_list in enumerate(train_df['go_terms']):

    # go_list is something like: ['GO:0003953', 'GO:0007165', 'GO:0019677', ...]
    #
    # Instead of checking every possible label (which would be slow),
    # we only iterate through the GO terms that actually exist, for this protein.

    for go in go_list:

        # We only care about GO terms that are inside our selected label set (top_go_terms).
        #
        # go_to_index is a dictionary:
        #   key   → GO term string, value → integer column index
        #
        # Dictionary lookup is O(1) (constant time),
        # which makes this approach very efficient.

        if go in go_to_index:

            # Retrieve the column index corresponding to this GO term.
            label_idx = go_to_index[go]

            # We now update the binary matrix.
            #
            # protein_idx → row position (which protein)
            # label_idx   → column position (which GO term)
            #
            # Setting value to 1 indicates that this protein is annotated with this GO term.

            binary_matrix[protein_idx, label_idx] = 1

# At this point, binary_matrix is a NumPy array.
# While NumPy arrays are memory efficient, they do not carry column names.
# For interpretability and easier downstream processing (saving to CSV, debugging, inspecting specific labels), we convert it into a Pandas DataFrame.

# We create column names using the original GO term list.
# Each column will be named:  label_GO:000xxxx
#
# This makes it clear that these columns represent binary label indicators.

column_names = [f"label_{go}" for go in top_go_terms]

# Now we create the DataFrame.
# Each column corresponds exactly to the index position defined earlier in go_to_index.

binary_df = pd.DataFrame(binary_matrix, columns=column_names)

# The shape of this DataFrame is: (number_of_proteins, number_of_selected_GO_terms)
# Example:
# Protein 1 → [0, 1, 0, 0, 1, ...]
# Protein 2 → [1, 0, 0, 1, 0, ...]
#
# This structure is known as a MULTI-LABEL BINARY MATRIX.
# It is the standard format required for multi-label classification.

print(f"Created binary DataFrame with {binary_df.shape[1]:,} label columns")

del binary_matrix
gc.collect()
mem3 = print_memory_usage("After binary matrix creation")

print(f"Matrix filling time: {time.time() - fill_start_time:.1f} seconds")
print(f"Total time elapsed: {time.time() - start_time:.1f} seconds")

Total proteins: 105,951
Total selected GO terms: 2,761
Created binary DataFrame with 2,761 label columns
After binary matrix creation - Memory Usage: 0.69 GB
Matrix filling time: 1.7 seconds
Total time elapsed: 12.0 seconds


In [10]:
# At this stage:
# - train_df contains original protein metadata (sequence, accession, etc.)
# - binary_df contains the multi-label binary matrix
#
# Both DataFrames have the same number of rows and are aligned in the same order.
# Then we concatenate them horizontally, axis=1 means column-wise concatenation.

final_df = pd.concat(
    [
        train_df[['sequence', 'accession']],  # original features
        binary_df                              # binary label matrix
    ], axis=1
)
print(f"Final DataFrame shape: {final_df.shape[0]:,} proteins × {final_df.shape[1]:,} columns")

# After concatenation, binary_df is no longer needed, Deleting it frees memory immediately.
del binary_df
gc.collect()

# We verify that:
# 1. Accession ID is correct
# 2. Sequence exists and has realistic length
# 3. Label columns exist and are correctly counted

print("\nVERIFICATION:")

example_accession = final_df.iloc[0]['accession']
example_sequence = final_df.iloc[0]['sequence']

print(f"Example protein: {example_accession}")
print(f"Sequence length: {len(example_sequence):,}")

# Count label columns
label_columns = [col for col in final_df.columns if col.startswith('label_')]
print(f"Number of GO term labels: {len(label_columns):,}")

# For multi-label classification, each protein can have multiple active labels.
# We sum across label columns for the first protein.
# Because labels are 0/1, sum gives the count of positive labels.

positive_counts = final_df.loc[0, label_columns].sum()
print(f"Positive labels for first protein: {positive_counts} out of {len(label_columns)}")

# A label column should not be entirely zero.
# That would mean no protein contains that GO term, which would make the label useless for training.

empty_labels = [col for col in label_columns if final_df[col].sum() == 0]

if empty_labels:
    print(f"Warning: {len(empty_labels)} labels have no positive examples!")
else:
    print("All labels have at least one positive example")

mem4 = print_memory_usage("After final dataframe")
print(f"Total time elapsed: {time.time() - start_time:.1f} seconds")

Final DataFrame shape: 105,951 proteins × 2,763 columns

VERIFICATION:
Example protein: A0A009IHW8
Sequence length: 269
Number of GO term labels: 2,761
Positive labels for first protein: 2 out of 2761
All labels have at least one positive example
After final dataframe - Memory Usage: 0.64 GB
Total time elapsed: 17.7 seconds


In [11]:
# HuggingFace Datasets is built on Apache Arrow, which is memory-efficient and columnar.
# Instead of manually chunking and building dictionaries, we directly convert the full DataFrame.
#
# preserve_index=False prevents the pandas index from becoming an additional column in the dataset.

hf_start_time = time.time()

dataset = Dataset.from_pandas(final_df, preserve_index=False)
print(f"Dataset created with {len(dataset):,} proteins")

# Count label columns
label_columns = [col for col in dataset.column_names if col.startswith('label_')]
print(f"Number of label columns: {len(label_columns):,}")

print(f"Conversion time: {time.time() - hf_start_time:.1f} seconds")

save_path = "C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/processed/protein_go_dataset"
print(f"Saving HuggingFace Dataset to: {save_path}")

# save_to_disk stores dataset in Arrow format
# This is much faster to reload later compared to CSV.

dataset.save_to_disk(save_path)
print("Dataset saved successfully.")

# Metadata helps us:
# - Track label frequency, Know which labels were frequent vs random, Compute imbalance statistics later

metadata = pd.DataFrame({
    'go_term': top_go_terms,
    'count': [go_counts[term] for term in top_go_terms],
    'percentage': [(go_counts[term] / len(final_df)) * 100 for term in top_go_terms],
    'is_frequent': [True] * len(top_go_frequent) + [False] * len(top_go_random)
})

metadata_path = "C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/processed/go_terms_metadata.csv"
metadata.to_csv(metadata_path, index=False)
print(f"Saved metadata to: {metadata_path}")

total_time = time.time() - start_time
final_mem = print_memory_usage("Final")

print("\nPERFORMANCE SUMMARY:")
print(f"Total processing time: {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
print(f"Memory increase: {final_mem - initial_mem:.2f} GB")
print(f"Time: {time.strftime('%H:%M:%S')}")

print("\nDATA SUMMARY:")
print(f"  • Input proteins: {len(train_df):,}")
print(f"  • Unique GO terms (original): {len(all_go_terms):,}")
print(f"  • Selected GO terms: {len(top_go_terms):,}")
print(f"  • Final dataset size: {final_df.shape[0]:,} × {final_df.shape[1]:,}")
print(f"  • Label representation: multi-hot binary (int8)")

print("\nAll steps completed successfully!")

Dataset created with 105,951 proteins
Number of label columns: 2,761
Conversion time: 28.5 seconds
Saving HuggingFace Dataset to: C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/processed/protein_go_dataset


Saving the dataset (1/1 shards): 100%|██████████| 105951/105951 [31:55<00:00, 55.31 examples/s]

Dataset saved successfully.
Saved metadata to: C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/processed/go_terms_metadata.csv
Final - Memory Usage: 0.41 GB

PERFORMANCE SUMMARY:
Total processing time: 1962.2 seconds (32.7 minutes)
Memory increase: 0.12 GB
Time: 23:26:58

DATA SUMMARY:
  • Input proteins: 105,951
  • Unique GO terms (original): 27,615
  • Selected GO terms: 2,761
  • Final dataset size: 105,951 × 2,763
  • Label representation: multi-hot binary (int8)

All steps completed successfully!


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from datasets import load_from_disk

print("✓ All libraries imported successfully")
print_memory_usage("After imports")

def set_seed(seed=42):
    """
    We fix all major random generators to ensure reproducibility.
    This ensures:
    - Same weight initialization
    - Same dataset shuffling
    - Same training behavior across runs
    """
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # If later using GPU: torch.cuda.manual_seed_all(seed)
    print(f"✓ Random seed set to: {seed}")

set_seed(42)
print_memory_usage("After setting seeds")

print("Loading dataset...")
mem_before = print_memory_usage("Before loading")
dataset_path = "C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/processed/protein_go_dataset"

# load_from_disk loads Arrow-formatted dataset efficiently.
# This does NOT load everything as Python objects.
# It keeps the memory-efficient Arrow backend.

dataset = load_from_disk(dataset_path)
print(f"✓ Loaded: {len(dataset):,} proteins")

mem_after = print_memory_usage("After loading")
print(f"✓ Memory used: {mem_after - mem_before:.3f} GB")

# Our dataset contains:
# - sequence
# - accession
# - 2700+ label columns (label_GO:XXXXXXX)
#
# We programmatically detect label columns instead of hardcoding them.

label_columns = [col for col in dataset.column_names if col.startswith("label_")]
print(f"✓ Number of GO term labels: {len(label_columns):,}")

# HuggingFace allows us to return tensors directly.
#
# type="torch" ensures:
# - label columns are returned as torch.Tensor
# - no pandas conversion
# - no duplication in RAM
#
# We include:
# - sequence (still string for now)
# - all label columns (as tensors)

dataset.set_format(
    type="torch",
    columns=label_columns  # labels as tensors
)
print("✓ Dataset formatted for PyTorch (labels as tensors)")

✓ All libraries imported successfully
After imports - Memory Usage: 0.47 GB
✓ Random seed set to: 42
After setting seeds - Memory Usage: 0.47 GB
Loading dataset...
Before loading - Memory Usage: 0.47 GB
✓ Loaded: 105,951 proteins
After loading - Memory Usage: 0.61 GB
✓ Memory used: 0.138 GB
✓ Number of GO term labels: 2,761
✓ Dataset formatted for PyTorch (labels as tensors)


In [13]:
import numpy as np
from datasets import load_from_disk

dataset_path = "C:/Users/USER/Documents/cod3astro/ML_AI/ProteinSeq_DL/data/processed/protein_go_dataset"
dataset = load_from_disk(dataset_path)

print("✓ Dataset loaded")
print(f"Total proteins: {len(dataset)}")

# Compute sequence lengths
seq_lengths = [len(seq) for seq in dataset["sequence"]]

# Convert to numpy for stats
seq_lengths = np.array(seq_lengths)

print("\n📊 Sequence Length Statistics")
print(f"Min length: {seq_lengths.min()}")
print(f"Max length: {seq_lengths.max()}")
print(f"Mean length: {seq_lengths.mean():.2f}")
print(f"Median length: {np.median(seq_lengths)}")
print(f"Std deviation: {seq_lengths.std():.2f}")

✓ Dataset loaded
Total proteins: 105951

📊 Sequence Length Statistics
Min length: 2
Max length: 35213
Mean length: 472.26
Median length: 358.0
Std deviation: 535.95


In [ ]:
# Create bins every 100 residues up to 2000
bins = list(range(0, 2000, 250))  # 0,250,500,...,2000
bins.append(float("inf"))         # >2000 bin

hist, bin_edges = np.histogram(seq_lengths, bins=bins)

print("\n📦 Sequence Length Distribution")
for i in range(len(hist)):
    start = int(bin_edges[i])
    end = int(bin_edges[i+1]) if bin_edges[i+1] != float("inf") else "∞"
    print(f"{start:>4} - {end:<4} : {hist[i]} proteins")


📦 Sequence Length Distribution
   0 - 250  : 35261 proteins
 250 - 500  : 37285 proteins
 500 - 750  : 17379 proteins
 750 - 1000 : 7145 proteins
1000 - 1250 : 3509 proteins
1250 - 1500 : 1878 proteins
1500 - 1750 : 956 proteins
1750 - ∞    : 2538 proteins


In [ ]:
print("\n🔍 Checking for duplicate sequences...")

unique_sequences = set(dataset["sequence"]) # Sequences that occurs once
total_sequences = len(dataset) # Total number of all sequences
unique_count = len(unique_sequences) # Total number of unoque sequence

duplicates = total_sequences - unique_count

print(f"Total sequences: {total_sequences}")
print(f"Unique sequences: {unique_count}")
print(f"Duplicate sequences: {duplicates}")


🔍 Checking for duplicate sequences...
Total sequences: 105951
Unique sequences: 102750
Duplicate sequences: 3201


In [17]:
print("🔍 Removing duplicate sequences (Arrow-safe method)...")

original_size = len(dataset)
print(f"Original dataset size: {original_size}")

# Get sequence column
sequences = dataset["sequence"]

# Find first occurrence index of each sequence
seen = {}
keep_indices = []

for idx, seq in enumerate(sequences):
    if seq not in seen:
        seen[seq] = True
        keep_indices.append(idx)

print(f"Keeping {len(keep_indices)} unique sequences")

# Select only first occurrences
dataset = dataset.select(keep_indices)

new_size = len(dataset)
print(f"New dataset size: {new_size}")
print(f"Removed duplicates: {original_size - new_size}")

🔍 Removing duplicate sequences (Arrow-safe method)...
Original dataset size: 105951
Keeping 102750 unique sequences
New dataset size: 102750
Removed duplicates: 3201


In [18]:
class ProteinDataset(Dataset):
    """
    This class prepares protein sequences and GO term labels for training in PyTorch.
    """

    def __init__(self, hf_dataset, max_seq_length=1024):
        """
        Args:
            hf_dataset: HuggingFace Arrow dataset
            max_seq_length: maximum allowed sequence length
        """

        # Store dataset reference
        self.dataset = hf_dataset
        self.max_seq_length = max_seq_length

        # Identify GO label columns automatically
        self.label_cols = [
            col for col in hf_dataset.column_names
            if col.startswith("label_")
        ]

        # Define amino acid vocabulary (20 standard amino acids)
        self.amino_acids = "ACDEFGHIKLMNPQRSTVWY"

        # Create mapping: amino acid → integer index
        # 0 reserved for padding / unknown
        self.aa_to_idx = {
            aa: i + 1 for i, aa in enumerate(self.amino_acids)
        }
        print(f"  Found {len(self.label_cols)} GO term labels")

    # Return total number of proteins
    def __len__(self):
        return len(self.dataset)

    # Get one protein sample
    def __getitem__(self, idx):

        # Get raw sequence
        sequence = self.dataset[idx]["sequence"]

        # Encode sequence to numeric form
        encoded_seq = self.encode_sequence(sequence)

        # Extract GO term labels
        labels = [
            self.dataset[idx][col]
            for col in self.label_cols
        ]
        # Convert to tensors
        return {
            "sequence": torch.tensor(encoded_seq, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.float32)
        }
    
    # Convert amino acid letters → numeric indices
    def encode_sequence(self, sequence):

        # Truncate if sequence is longer than max length
        sequence = sequence[:self.max_seq_length]

        # Convert each amino acid to number
        encoded = [
            self.aa_to_idx.get(aa, 0)  # 0 if unknown
            for aa in sequence
        ]

        # Pad with zeros if sequence is shorter
        padding_length = self.max_seq_length - len(encoded)
        if padding_length > 0:
            encoded += [0] * padding_length

        return encoded

print("✓ ProteinDataset class created successfully")
print_memory_usage("After creating dataset class")

✓ ProteinDataset class created successfully
After creating dataset class - Memory Usage: 0.27 GB


0.27440643310546875

In [19]:
mem_before = print_memory_usage("Before splitting data")

# STEP 1: Split into 70% train and 30% temporary
split_1 = dataset.train_test_split(
    test_size=0.3,
    seed=42
)

train_dataset_arrow = split_1["train"]
temp_dataset_arrow = split_1["test"]

# STEP 2: Split temporary 30% into 15% val + 15% test
split_2 = temp_dataset_arrow.train_test_split(
    test_size=0.5,
    seed=42
)

val_dataset_arrow = split_2["train"]
test_dataset_arrow = split_2["test"]

# Wrap each split with our PyTorch Dataset class
train_dataset = ProteinDataset(train_dataset_arrow, max_seq_length=256)
val_dataset = ProteinDataset(val_dataset_arrow, max_seq_length=256)
test_dataset = ProteinDataset(test_dataset_arrow, max_seq_length=256)

print("✓ Data split completed:")
print(f"  Training samples: {len(train_dataset):,}")
print(f"  Validation samples: {len(val_dataset):,}")
print(f"  Test samples: {len(test_dataset):,}")

# Memory tracking
mem_after = print_memory_usage("After splitting data")
print(f"  Memory used by splits: {mem_after - mem_before:.2f} GB")

gc.collect()
print("  Garbage collection performed")
print_memory_usage("After garbage collection")

Before splitting data - Memory Usage: 0.05 GB
  Found 2761 GO term labels
  Found 2761 GO term labels
  Found 2761 GO term labels
✓ Data split completed:
  Training samples: 71,925
  Validation samples: 15,412
  Test samples: 15,413
After splitting data - Memory Usage: 0.46 GB
  Memory used by splits: 0.42 GB
  Garbage collection performed
After garbage collection - Memory Usage: 0.46 GB


0.4638175964355469

In [20]:
# MODEL DEFINITION

# In this section, we define a Convolutional Neural Network (CNN) for multi-label protein function prediction.
# The goal is to take an amino acid sequence and predict multiple GO term labels simultaneously.

class SimpleProteinCNN(nn.Module):
    """
    Convolutional Neural Network for multi-label GO term prediction.

    Architecture Overview:
    1. Embedding layer (converts amino acids to dense vectors)
    2. 3 Convolutional layers (extract sequence patterns)
    3. Global Max Pooling (captures strongest motif signals)
    4. Fully connected layers (combine extracted features)
    5. Output layer (one neuron per GO term)
    """

    def __init__(self, num_classes):
        super().__init__()

        # EMBEDDING LAYER

        # We convert amino acid indices into dense vectors.
        #
        # Neural networks do not work well with raw integers.
        # Embeddings allow the model to learn biochemical similarity between amino acids (e.g., hydrophobic residues may learn similar vector representations).
        #
        # 21 input tokens: 20 standard amino acids, 1 padding/unknown token (index 0)
        # 64 = embedding dimension

        self.embedding = nn.Embedding(
            num_embeddings=21,
            embedding_dim=64,
            padding_idx=0
        )

        # CONVOLUTIONAL FEATURE EXTRACTORS

        # We use 3 Conv1D layers with increasing receptive fields.
        #
        # Conv1: kernel_size=3 → captures short motifs (tri-peptides)
        # Conv2: kernel_size=5 → medium motifs
        # Conv3: kernel_size=7 → longer motifs
        #
        # Padding ensures output length stays same as input length.

        self.conv_layers = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv1d(128, 64, kernel_size=5, padding=2),
            nn.ReLU(),

            nn.Conv1d(64, 32, kernel_size=7, padding=3),
            nn.ReLU()
        )

        # GLOBAL MAX POOLING

        # AdaptiveMaxPool1d(1) reduces sequence dimension to 1.
        #
        # This means:
        # From (batch, channels, seq_len)    To   (batch, channels, 1)
        #
        # It selects the strongest activation across the sequence,
        # meaning it keeps the most important detected motif.

        self.global_pool = nn.AdaptiveMaxPool1d(1)

        # FULLY CONNECTED CLASSIFIER

        # After pooling, we reduce feature space and learn nonlinear label interactions.
        #
        # Dropout prevents overfitting by randomly disabling neurons during training.

        self.classifier = nn.Sequential(
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, num_classes)
        )

    # FORWARD PASS

    # This defines how input flows through the network.

    def forward(self, x):
        """
        Input:
            x → shape (batch_size, sequence_length)
        Output:
            logits → shape (batch_size, num_classes)
        """

        # STEP 1: Embedding
        # Input:  (batch, seq_len)
        # Output: (batch, seq_len, 64)
        x = self.embedding(x)

        # STEP 2: Rearranging for Conv1D
        # Conv1D expects: (batch, channels, seq_len), So we permute dimensions.
        x = x.permute(0, 2, 1)

        # STEP 3: Convolutional feature extraction
        x = self.conv_layers(x)

        # STEP 4: Global pooling
        # (batch, 32, seq_len) → (batch, 32, 1)
        x = self.global_pool(x)

        # Remove last dimension → (batch, 32)
        x = x.squeeze(-1)

        # STEP 5: Classification
        logits = self.classifier(x)
        return logits

print("✓ SimpleProteinCNN class created")


# PARAMETER COUNT
# Counting parameters helps us understand:
# - Model complexity, Risk of overfitting, Memory requirements

sample_model = SimpleProteinCNN(num_classes=len(train_dataset.label_cols))
total_params = sum(p.numel() for p in sample_model.parameters())

print(f"  Model parameters: {total_params:,}")
print_memory_usage("After creating model")

✓ SimpleProteinCNN class created
  Model parameters: 273,385
After creating model - Memory Usage: 0.02 GB


0.0248870849609375

In [21]:
# DEVICE SETUP
# In this section, we define the computational device.
# Since we are working on a laptop without GPU configuration, we explicitly select CPU to avoid CUDA-related issues.

print("Step 1: Setting up device...")
device = torch.device("cpu")

print(f"✓ Using device: {device}")
print_memory_usage("After device setup")

# VERIFY DATASET STRUCTURE
# Before training, we confirm: The dataset size, The number of GO term labels, That label columns exist
# This ensures that our dataset class was constructed correctly.

print("\nStep 2: Verifying dataset...")
print(f"Training proteins: {len(train_dataset)}")
print(f"Validation proteins: {len(val_dataset)}")
print(f"Test proteins: {len(test_dataset)}")

if hasattr(train_dataset, "label_cols"):
    num_classes = len(train_dataset.label_cols)
    print(f"✓ Found {num_classes} GO term labels")
else:
    raise ValueError("Dataset does not contain label columns.")


# CREATE DATA LOADERS
# DataLoader:
# - Loads data in batches, Handles shuffling, Improves memory efficiency
# num_workers=0 is required for Windows to avoid multiprocessing issues.

print("\nStep 3: Creating DataLoaders...")

batch_size = 8  # Small batch size for CPU memory safety

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

print(f"✓ DataLoaders created")
print(f"  Batch size: {batch_size}")
print(f"  Training batches: {len(train_loader)}")


# DEFINE MODEL ARCHITECTURE
# We define a lightweight CNN suitable for CPU training.
# This version is intentionally simple to: Reduce memory usage, Reduce computation time, Ensure stability on laptops

print("\nStep 4: Creating model...")

class LightweightProteinCNN(nn.Module):
    
   # Simplified CNN for multi-label protein function prediction.

    def __init__(self, num_classes):
        super().__init__()

        # Embedding Layer
        # Converts amino acid indices into 32-dimensional vectors.
        # 21 tokens = 20 amino acids + padding.
        self.embedding = nn.Embedding(
            num_embeddings=21,
            embedding_dim=32,
            padding_idx=0
        )

        # Convolution Layer
        # Detects short sequence motifs (kernel_size=3).
        self.conv = nn.Conv1d(
            in_channels=32,
            out_channels=64,
            kernel_size=3,
            padding=1
        )

        # Global Max Pooling
        # Reduces sequence dimension to 1.
        self.pool = nn.AdaptiveMaxPool1d(1)

        # Output Layer
        # One neuron per GO term.
        self.fc = nn.Linear(64, num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):

        # Step 1: Embedding
        # (batch, seq_len) → (batch, seq_len, 32)
        x = self.embedding(x)

        # Step 2: Rearrange for Conv1D
        # (batch, seq_len, 32) → (batch, 32, seq_len)
        x = x.permute(0, 2, 1)

        # Step 3: Convolution + Activation
        x = self.relu(self.conv(x))

        # Step 4: Global Pooling
        # (batch, 64, seq_len) → (batch, 64, 1)
        x = self.pool(x)

        # Remove last dimension
        x = x.squeeze(-1)

        # Step 5: Output layer (logits)
        x = self.fc(x)

        return x


# Instantiate model
model = LightweightProteinCNN(num_classes=num_classes).to(device)

print(f"✓ Model created: {model.__class__.__name__}")

total_params = sum(p.numel() for p in model.parameters())
print(f"  Total parameters: {total_params:,}")


# DEFINE LOSS FUNCTION AND OPTIMIZER
# Since this is multi-label classification, we use BCEWithLogitsLoss (NOT CrossEntropyLoss).
#
# BCEWithLogitsLoss: Applies sigmoid internally, Computes binary cross-entropy for each label independently

print("\nStep 5: Setting up loss and optimizer...")

criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(
    model.parameters(),
    lr=0.001,
    weight_decay=0.0
)

print("✓ Loss function and optimizer initialized")


# QUICK SANITY CHECK
# Before real training, we verify:
# 1. DataLoader returns correct shapes
# 2. Model forward pass works
# 3. Loss computation works
# 4. Backpropagation works

print("\nStep 6: Running sanity check...")

batch = next(iter(train_loader))

sequences = batch["sequence"].to(device)
labels = batch["labels"].to(device)

print(f"Batch shapes:")
print(f"  Sequences: {sequences.shape}")
print(f"  Labels: {labels.shape}")

# Forward pass
outputs = model(sequences)
print(f"  Model outputs: {outputs.shape}")

# Loss computation
loss = criterion(outputs, labels)
print(f"  Loss: {loss.item():.4f}")

# Backpropagation test
optimizer.zero_grad()
loss.backward()
optimizer.step()

print("✓ Forward and backward pass successful")
print_memory_usage("After training setup")


# FINAL SUMMARY

print("\n" + "="*50)
print("FINAL TRAINING SETUP SUMMARY")
print("="*50)

print(f"Device: {device}")
print(f"Model: {model.__class__.__name__}")
print(f"Parameters: {total_params:,}")
print(f"Optimizer: {optimizer.__class__.__name__}")
print(f"Batch size: {batch_size}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

print("\nREADY FOR TRAINING")
print("="*50)

Step 1: Setting up device...
✓ Using device: cpu
After device setup - Memory Usage: 0.02 GB

Step 2: Verifying dataset...
Training proteins: 71925
Validation proteins: 15412
Test proteins: 15413
✓ Found 2761 GO term labels

Step 3: Creating DataLoaders...
✓ DataLoaders created
  Batch size: 8
  Training batches: 8991

Step 4: Creating model...
✓ Model created: LightweightProteinCNN
  Total parameters: 186,345

Step 5: Setting up loss and optimizer...
✓ Loss function and optimizer initialized

Step 6: Running sanity check...
Batch shapes:
  Sequences: torch.Size([256])
  Labels: torch.Size([256, 8])


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3